# **KDDM 2 SARTORIUS CELL INSTANCE SEGMENTATION U-NET TRAIN AND INFERENCE NOTEBOOK**

# INSTALL All DEPENDENCIES
* **Install segmentation_models.pytorch and required dependencies**
* **Import libaries**
* **Helper function to visualize images**

In [ ]:
!cp -r /kaggle/input/kddm2/unet /kaggle/working/carrotssmp

!pip install /kaggle/working/carrotssmp/timm-0.4.12/timm-0.4.12-py3-none-any.whl --no-deps
!cd /kaggle/working/carrotssmp/efficientnet_pytorch-0.6.3/ && pip install -e .
!cd /kaggle/working/carrotssmp/pretrainedmodels-0.7.4/ && pip install -e .
!pip install /kaggle/working/carrotssmp/segmentation_models.pytorch-0.2.1/segmentation_models_pytorch-0.2.1-py3-none-any.whl --no-deps

import sys
sys.path.append("/kaggle/working/carrotssmp/efficientnet_pytorch-0.6.3/")
sys.path.append("/kaggle/working/carrotssmp/pretrainedmodels-0.7.4/")

In [ ]:
import cupy as cp
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

import albumentations as albu

from scipy.ndimage.measurements import label
import segmentation_models_pytorch as smp
import shutil
import random
import torch

In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

# DATALOADER
* **Read annotation csv and generate mask immages**


In [ ]:
SARTORIUS_CLASSES = ['astro', 'cort', 'shsy5y']

#define path to input image folder, annotation csv and output folders for images and masks
img_dir = "/kaggle/input/sartorius-cell-instance-segmentation/train"
annotation_file = "/kaggle/input/sartorius-cell-instance-segmentation/train.csv"
x_train_dir = "/kaggle/working/train"
y_train_dir = "/kaggle/working/trainannot"
x_val_dir = "/kaggle/working/val"
y_val_dir = "/kaggle/working/valannot"

os.makedirs(x_train_dir, exist_ok=True)
os.makedirs(y_train_dir, exist_ok=True)
os.makedirs(x_val_dir, exist_ok=True)
os.makedirs(y_val_dir, exist_ok=True)

#random train/val split
percentage_in_train = 0.9
frames = os.listdir(img_dir)
num_train = int(len(frames) * percentage_in_train)
random.seed(0)
train_frames = set( random.sample(frames, num_train) )

with open(annotation_file, "r") as f:
    anns = f.read().split("\n")
anns = anns[1:-1]
print("annotations", len(anns))

#create dict with annotation masks per img and convert rle to "start to end" pixel masks
anns_per_img_id = {}
for a in anns:
    values = a.split(",")
    id = values[0]
    cell_type = values[4]
    rle_mask = values[1].split(" ")

    starts = list(map(lambda x: int(x) - 1, rle_mask[0::2]))
    lengths = list(map(int, rle_mask[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]

    if id not in anns_per_img_id.keys():
        anns_per_img_id[id] = {}
        anns_per_img_id[id]["start_end"] = []

    anns_per_img_id[id]["start_end"].append( [starts, ends] )
    anns_per_img_id[id]["cell_type"] = cell_type
num_imgs_keys = len(anns_per_img_id.keys())
print("anns_per_img_id", num_imgs_keys)


#generate image masks
pixel_val_per_class = {'astro': 1, 'cort': 2, 'shsy5y': 3}
img_mask = np.empty((704*520), dtype=np.uint8)
img_mask_2d = np.empty((520, 704), dtype=np.uint8)
for i in tqdm( range(num_imgs_keys) ):
    k = list(anns_per_img_id.keys())[i]
    anns = anns_per_img_id[k]
    masks = anns["start_end"]
    id = anns["cell_type"]
    img_mask.fill(0)
    for m in masks:
        for start, end in zip(m[0], m[1]):
            img_mask[start:end] = pixel_val_per_class[id]
    img_mask_2d = img_mask.reshape((520, 704))
    fname = "{:s}.png".format(k)
    if fname in train_frames:
        shutil.copy( os.path.join(img_dir, fname), os.path.join(x_train_dir, fname) )
        cv2.imwrite( os.path.join(y_train_dir, fname), img_mask_2d )
    else:
        shutil.copy( os.path.join(img_dir, fname), os.path.join(x_val_dir, fname) )
        cv2.imwrite( os.path.join(y_val_dir, fname), img_mask_2d )
    
print( "train imgs:", len(os.listdir(x_train_dir)), "train mask imgs:", len(os.listdir(y_train_dir)) )
print( "val imgs:", len(os.listdir(x_val_dir)), "val mask imgs:", len(os.listdir(y_val_dir)) )

* **Class for loading the dataset and applying augmentations/preprocessing**

In [ ]:
class Dataset(BaseDataset):
    """Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = SARTORIUS_CLASSES
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask
        
    def __len__(self):
        return len(self.ids)
    
    def getItemPerFname(self, fname):
        # read data
        image = cv2.imread(fname)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = np.zeros((image.shape[0], image.shape[1]))
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask
        

* **Load train dataset**

In [ ]:
dataset = Dataset(x_train_dir, y_train_dir, classes=SARTORIUS_CLASSES)

image, mask = dataset[5] # get some sample
visualize(
    image=image, 
    cell_masks=mask.squeeze(),
)

* **Define augmentations**

In [ ]:
def get_training_augmentation():
    train_transform = [
        albu.HorizontalFlip(p=0.5),
        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),
        albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),
        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(544, 702)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

# **TRAINING**
* **Set settings for training**
* **Execute training**

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/kddm2/unet/resnext50_32x4d-7cdf4587.pth /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
!ls /root/.cache/torch/hub/checkpoints/

* **Define model, activation function etc. and initialize epoch runners**

In [ ]:
ENCODER = 'resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'


# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(SARTORIUS_CLASSES), 
    activation=ACTIVATION,
)
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)


#define loss function and optimizer
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.8),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])


#define datasets and data loader
train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=SARTORIUS_CLASSES,
)

valid_dataset = Dataset(
    x_val_dir, 
    y_val_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=SARTORIUS_CLASSES,
)
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=2)


#create epoch runners (simple loop of iterating over dataloader`s samples)
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

* **Train model**

In [ ]:
out_dir = "/kaggle/working/results/"
os.makedirs(out_dir, exist_ok=True)

max_score = 0
max_epochs = 30
# train model for 
for i in range(0, max_epochs):
    print('\nEpoch: {}'.format(i+1))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']: #todo
        max_score = valid_logs['iou_score'] #todo
        torch.save(model, out_dir + "best_model.pth")
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')

# **INFERENCE**
* **Perform inference on the test dataset**
* **Output sample images for visualization**
* **Store in kaggle competition submission format**

In [ ]:
def mask2rle(msk):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    msk    = cp.array(msk)
    pixels = msk.flatten()
    pad    = cp.array([0])
    pixels = cp.concatenate([pad, pixels, pad])
    runs   = cp.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


x_test_dir = "/kaggle/input/sartorius-cell-instance-segmentation/test"
y_test_dir = "/kaggle/working/testannot"
os.makedirs(y_test_dir, exist_ok=True)

dummy_test_mask = np.zeros((520, 704), dtype=np.uint8)
for f in os.listdir(x_test_dir):
    cv2.imwrite( os.path.join(y_test_dir, f), dummy_test_mask )

test_dataset = Dataset(
    x_test_dir, 
    y_test_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=SARTORIUS_CLASSES,
)

test_dataset_vis = Dataset(
    x_test_dir,
    y_test_dir, 
    classes=SARTORIUS_CLASSES,
)

with open(out_dir + "best_model.pth", "rb") as f:
    best_model = torch.load(f)

#run inference on test files and store submissions
output = {"rles": [], "ids": []}
test_files = sorted(os.listdir(x_test_dir))
for test_file in test_files:
    #load file
    id = test_file[:-4]
    file_path = os.path.join(x_test_dir, test_file)
    image_vis = test_dataset_vis.getItemPerFname(file_path)[0].astype('uint8')
    image = test_dataset.getItemPerFname(file_path)[0]
    
    
    #perform inference on image
    x_tensor = torch.from_numpy(image).to(DEVICE)[None]
    pr_mask = best_model.predict(x_tensor).detach()
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
    
    
    #visualize input image with mask overlay
    merged = np.zeros((520, 704, 3), dtype=np.uint8)
    for aa in range(merged.shape[0]):
        for bb in range(merged.shape[1]):
            if pr_mask[0, aa+12, bb] < 0.5:
                merged[aa, bb, 0] = 255
                merged[aa, bb, 1] = 5
                merged[aa, bb, 2] = 0
            else:
                merged[aa, bb, 0] = image_vis[aa, bb, 0]
                merged[aa, bb, 1] = image_vis[aa, bb, 1]
                merged[aa, bb, 2] = image_vis[aa, bb, 2]
    cv2.imwrite( os.path.join("/kaggle/working/", "{:s}_merged.png".format(id)), merged )
    
    
    #split mask into instance masks
    uint8_mask = (255 - pr_mask[0]*255).astype(np.uint8)[12:-12, :]
    labeled_colorized = np.zeros((520, 704, 3), dtype=np.uint8)
    structure = np.ones((3, 3), dtype=np.int)
    labeled, ncomponents = label(uint8_mask, structure)
    
    
    #visualize input image with instance masks overlay
    labels, counts = np.unique(labeled, return_counts=True)
    bg_label = labels[np.argmax(counts)]
    print("bg_label", bg_label)
    colors = np.random.choice(range(256), size=3*ncomponents)
    for aa in range(labeled_colorized.shape[0]):
        for bb in range(labeled_colorized.shape[1]):
            if labeled[aa, bb] == bg_label:
                continue
            c = 3*(labeled[aa, bb]-1)
            for i in range(3):
                labeled_colorized[aa, bb, i] = colors[c+i]
    plt.figure(figsize=(8, 8))
    plt.imshow(labeled_colorized)
    cv2.imwrite( os.path.join("/kaggle/working/", "{:s}_labeled.png".format(id)), labeled_colorized )
    
    
    #convert predictions to kaggle submission format
    for l in labels:
        single_instance_mask = np.zeros((520, 704))
        if l == bg_label:
            continue
        x_ind = np.where(labeled == l)[0]
        y_ind = np.where(labeled == l)[1]
        single_instance_mask[x_ind, y_ind] = 1
        rle_mask =  mask2rle(single_instance_mask)
        output["rles"].append(rle_mask)
        output["ids"].append(id)

#store predictions in kaggle submission format
indexes = []
for i, segm in enumerate(output["rles"]):
    if segm == '':
        indexes.append(i)
for element in sorted(indexes, reverse = True):
    del output["rles"][element]
    del output["ids"][element]
    
files = pd.Series(output["ids"], name='id')
preds = pd.Series(output["rles"], name='predicted')
submission_df = pd.concat([files, preds], axis=1)
print(submission_df.head())
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


for file in test_files:
    print("final masks in file '{:s}': {:d}".format(file, output["ids"].count(str(file.split('.')[0]))))

In [ ]:
!ls /kaggle/working
!echo ""
%cd /kaggle/working
!rm -rf /kaggle/working/data
!rm -rf /kaggle/working/train
!rm -rf /kaggle/working/trainannot
!rm -rf /kaggle/working/val
!rm -rf /kaggle/working/valannot
!rm -rf /kaggle/working/carrotssmp
!rm -rf /kaggle/working/results
!rm -rf /kaggle/working/testannot
!echo ""
!ls /kaggle/working